In [15]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras.layers import Input,Dense,Dropout
from tensorflow.keras.models import Model
import pickle

In [16]:
#Load the processed dataset
data=pd.read_csv("Final.csv")

In [17]:
#Separates the independent variables and dependent variable into x and y
columns = [x for x in data.columns if x not in ['Facebook_scaled','LinkedIn_scaled','GooglePlus_scaled']]
x = data[columns]
y = data[['Facebook_scaled']]

#Splits the dataset into 70 percent training set and 30 percent testing set
# keeping the random_state value same makes the function to split same rows into training and testing every time
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=30)

In [18]:
import statsmodels.api as sm
def bidirection(data, target,SL_in=0.05,SL_out = 0.05):
    initial = data.columns.tolist()
    final= []
    while (len(initial)>0):
        remaining = list(set(initial)-set(final))
        new_pval = pd.Series(index=remaining)
        for new_column in remaining:
            model = sm.OLS(target, sm.add_constant(data[final+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if(min_p_value<SL_in):
            final.append(new_pval.idxmin())
            while(len(final)>0):
                best_features_with_constant = sm.add_constant(data[final])
                p_values = sm.OLS(target, best_features_with_constant).fit().pvalues[1:]
                max_p_value = p_values.max()
                if(max_p_value >= SL_out):
                    excluded_feature = p_values.idxmax()
                    final.remove(excluded_feature)
                else:
                    break 
        else:
            break
    return final

In [19]:
#Perform the Bi-directional elimination
cols=bidirection(x,y)

C:\Users\Jack\AppData\Local\Temp\ipykernel_12840\3162902221.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining)
C:\Users\Jack\AppData\Local\Temp\ipykernel_12840\3162902221.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining)
C:\Users\Jack\AppData\Local\Temp\ipykernel_12840\3162902221.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining)
C:\Users\Jack\AppData\Local\Temp\ipykernel_12840\3162902221.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Spec

In [ ]:
#Only get the remaining features after elimination
features=x[cols]
features.head()

,gp_t3,fb_t1,fb_t3,gp_t2,gp_t1,fb_t4,Topic_obama,fb_t2,li_t4,gp_t4,li_t1,li_t3
0,0,0.0,0.0,0,0,0.0,0,0.0,0,0,0,0
1,0,0.0,0.0,0,0,0.0,0,0.0,0,0,0,0
2,0,0.0,0.0,0,0,0.0,0,0.0,0,0,0,0
3,0,0.0,0.0,0,0,0.0,0,0.0,0,0,0,0
4,0,0.0,0.0,0,0,0.0,0,0.0,0,0,0,0


In [ ]:
#save the remaining features after bi-directional elimination in to csv
features.to_csv('bi_variable_selection.csv',index=False)
y.to_csv('bi_variable_labels.csv',index=False)